# DRL usage example

First we import al the used libraries, remember to always import `Sinergym` even if it says is not used, because that is needed to define the environments

In [6]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
from datetime import datetime
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv

First let's define some strings and variables for the execution.

In [7]:
environment = "Eplus-demo-v1"
episodes = 4
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

# register run name
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"

Now we are ready to create the Gym Environment. Here we use the run name defined before as well as the type of reward, in our case we are going to use the LinearReward defined by `Sinergym`. You can define your own or use any of the other defined by `Sinergym` have a look at ref:`rewards` for more information on that.

In [8]:
env = gym.make(environment, reward=LinearReward)

[2022-05-16 13:33:59,775] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:33:59,780] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...


We can add also a Wrapper to the environment, we are going to use a Logger (extension of ``gym.Wrapper``) this is used to monitor and log the interactions with the environment and save the data into a CSV.

In [9]:
env = LoggerWrapper(env)

At this point we have the environment all set up and ready to be used to define and create our learning model in this case it's going to be a DQN, but we can use any other (have a look at the `DRL_battery.py` and read :ref:`Deep Reinforcement Learning Integration` for more detailed information on available DRL algorithms).
Please feel free to play and change the values of the attributes of our model (or even the model) to see the differences.

In [10]:
model = DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Now we need to calculate the number of timesteps of each episode for the evaluation.

In [11]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize


Now we need to create a vectorized wrapper for the environment because the callbacks we are going to use require a vector.

In [12]:
env_vec = DummyVecEnv([lambda: env])

We are going to use the LoggerEval callback to print and save the training process.

In [13]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    env_vec,
    best_model_save_path='best_model/' + name + '/',
    log_path='best_model/' + name + '/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=2)
callbacks.append(eval_callback)

callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [14]:
timesteps = episodes * n_timesteps_episode

Now is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [15]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

[2022-05-16 13:34:36,641] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:34:36,673] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run1
[2022-05-16 13:35:01,132] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:35:01,132] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:35:01,167] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run2


-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 3.5e+04   |
|    ep_rew_mean      | -2.24e+04 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 1         |
|    fps              | 1396      |
|    time_elapsed     | 25        |
|    total_timesteps  | 35040     |
-----------------------------------


[2022-05-16 13:35:40,865] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:35:40,867] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:35:40,912] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run3
/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
[2022-05-16 13:35:47,155] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:35:47,156] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:35:47,191] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:

Eval num_timesteps=70080, episode_reward=-19505.93 +/- 0.00
Episode length: 35040.00 +/- 0.00
New best mean reward!
----------------------------------------
| eval/                   |            |
|    comfort_penalty      | -2.14e+04  |
|    comfort_violation(%) | 35.7       |
|    mean_ep_length       | 3.5e+04    |
|    mean_power_consum... | 1.76e+08   |
|    mean_rewards         | -19505.926 |
|    power_penalty        | -1.76e+04  |
|    std_rewards          | 0.0        |
| rollout/                |            |
|    ep_len_mean          | 3.5e+04    |
|    ep_rew_mean          | -2.08e+04  |
|    exploration_rate     | 0.05       |
| time/                   |            |
|    episodes             | 2          |
|    fps                  | 538        |
|    time_elapsed         | 130        |
|    total_timesteps      | 70080      |
| train/                  |            |
|    learning_rate        | 0.0001     |
|    loss                 | 15.5       |
|    n_updates         

[2022-05-16 13:37:35,920] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:37:35,920] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:37:35,952] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run7


-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 3.5e+04   |
|    ep_rew_mean      | -1.97e+04 |
|    exploration_rate | 0.05      |
| time/               |           |
|    episodes         | 3         |
|    fps              | 584       |
|    time_elapsed     | 179       |
|    total_timesteps  | 105120    |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 18.7      |
|    n_updates        | 13779     |
-----------------------------------


[2022-05-16 13:38:28,273] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:38:28,273] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:38:28,305] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run8
[2022-05-16 13:38:34,736] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:38:34,737] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:38:34,771] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run9
[2022-05-16 13:39:05,465] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-05-16 13:39:05,465] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode

Eval num_timesteps=140160, episode_reward=-20375.97 +/- 0.00
Episode length: 35040.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    comfort_penalty      | -2.29e+04  |
|    comfort_violation(%) | 35.6       |
|    mean_ep_length       | 3.5e+04    |
|    mean_power_consum... | 1.78e+08   |
|    mean_rewards         | -20375.975 |
|    power_penalty        | -1.78e+04  |
|    std_rewards          | 0.0        |
| rollout/                |            |
|    ep_len_mean          | 3.5e+04    |
|    ep_rew_mean          | -2e+04     |
|    exploration_rate     | 0.05       |
| time/                   |            |
|    episodes             | 4          |
|    fps                  | 467        |
|    time_elapsed         | 300        |
|    total_timesteps      | 140160     |
| train/                  |            |
|    learning_rate        | 0.0001     |
|    loss                 | 8.5        |
|    n_updates            | 22539      |
---

Now we save the current model.

In [16]:
model.save(env.simulator._env_working_dir_parent + '/' + name)

And as always, remember to close the environment.

In [17]:
env.close()

[2022-05-16 13:39:42,683] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
